## Scenario 1
#### Problem Statement
- **The dataset consists of 2-Dimensional spectrograms of radio signals from space collected at the SETI Instituteby  the Allen  Telescope  Array.The  objective  is  to classify the radio signalsfrom outer spaceinto one of four classes. Dataset DescriptionSETI Dataset**

##### •Training Data: otrain_images: Normalized values of Pixels otrain_labels: Stored as One-Hot Encoded data
##### •Validation Data: oval_images: Normalized values of Pixels oval_labels: Stored as One-Hot Encoded data
##### •Classes: “squiggle”, “narrowband”, “narrowbanddrd”, and “noise”


### Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf

from sklearn.metrics import confusion_matrix
from livelossplot.tf_keras import PlotLossesCallback
from sklearn import metrics

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

import numpy as np
np.random.seed(42)
import warnings;warnings.simplefilter('ignore')
%matplotlib inline
print('Tensorflow version:', tf.__version__)

#### Load and Pre-process the dataset

### Load the dataset using the pandas read_csvfunction

In [ ]:
Train_Images = pd.read_csv(r"C:\Users\Shivani Dussa\Downloads\dataset\train\images.csv",header = None)
Train_Labels = pd.read_csv(r"C:\Users\Shivani Dussa\Downloads\dataset\train\labels.csv",header = None)

In [ ]:
val_Images = pd.read_csv(r"C:\Users\Shivani Dussa\Downloads\dataset\valid\images (1).csv",header = None)
val_Labels = pd.read_csv(r"C:\Users\Shivani Dussa\Downloads\dataset\valid\labels (1).csv",header = None)

In [ ]:
Train_Images.head()

In [ ]:
Train_Labels.head()

In [ ]:
val_Images.head()

In [ ]:
val_Labels.head()

### Check the shape of the training and validation data

In [ ]:
print("Training data shape:")
print(Train_Images.shape)
print(Train_Labels.shape)

In [ ]:
print("Validation set shape:")
print(val_Images.shape)
print(val_Labels.shape)

### Reshape the training and validation images

In [ ]:
x_train = Train_Images.values.reshape(3200, 64, 128, 1)
x_val = val_Images.values.reshape(800, 64, 128, 1)

y_train = Train_Labels.values
y_val = val_Labels.values

In [ ]:
x_train.shape

In [ ]:
x_val.shape

### Visualize the dataset

In [ ]:
plt.figure(0, figsize=(12,12))
for i in range(1,3):
    plt.subplot(1,2,i)
    img = np.squeeze(x_train[np.random.randint(0, x_train.shape[0])]) # np.squeeze is used to delete non  useful dimension in (64,128,1) and transform the shape into (64,128) in order to fit into plt.imshow
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img,cmap="gray")

In [ ]:
plt.figure(0, figsize=(12,12))
for i in range(1,5):
    plt.subplot(1,4,i)
    img = np.squeeze(x_train[np.random.randint(0, x_train.shape[0])]) # np.squeeze is used to delete non  useful dimension in (64,128,1) and transform the shape into (64,128) in order to fit into plt.imshow
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img,cmap="gray")

### Create Training and Validation Data Generators using Keras Image DataGenerator function

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

datagen_train = ImageDataGenerator(horizontal_flip=True)
datagen_train.fit(x_train)

datagen_val = ImageDataGenerator(horizontal_flip=True)
datagen_val.fit(x_val)

### Design a Convolutional Neural Network (CNN) Model

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

In [ ]:
model = Sequential()

# 1st CNN layer 
model.add(Conv2D(32,(5,5), padding='same', input_shape=(64, 128,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


# 2nd CNN layer
model.add(Conv2D(64,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


# Flattening
model.add(Flatten())

# Dense Layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(4, activation='softmax'))

### Compile the Modelusing Adam optimizer, categorical_crossentropy loss function, and accuracy metric

In [ ]:
initial_learning_rate = 0.005
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,decay_steps=5,decay_rate=0.96,staircase=True)   

optimizer = Adam(learning_rate=lr_schedule)

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Print the Model summary

In [ ]:
model.summary()

### Train the Modelwith batch_size = 32 & epochs = 12

In [ ]:
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_loss',
                             save_weights_only=True, mode='min', verbose=0)
callbacks = [PlotLossesCallback(), checkpoint]
batch_size = 32

In [ ]:
history = model.fit(datagen_train.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
    steps_per_epoch=len(x_train)//batch_size,
    validation_data = datagen_val.flow(x_val, y_val, batch_size=batch_size, shuffle=True),
    validation_steps = len(x_val)//batch_size,
    epochs=12,
    callbacks=checkpoint
)

### Evaluate the Model

#### Use the model.evaluatefunction to evaluate the accuracy

In [ ]:
model.evaluate(x_val, y_val)

#### Print a Classification Reportand the accuracy score (classification accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import seaborn as sns

y_true = np.argmax(y_val, 1)
y_pred = np.argmax(model.predict(x_val), 1)
print(metrics.classification_report(y_true, y_pred))

In [ ]:
print("Classification accuracy: %0.6f" % metrics.accuracy_score(y_true, y_pred))

- The accuracy is 74% 

#### Display a Confusion Matrix to evaluate the performance of the model

In [ ]:
print(confusion_matrix(y_true,y_pred))

In [ ]:
labels =  ['squiggle', 'narrowband', 'narrowbanddrd','noise']

ax= plt.subplot()
sns.heatmap(metrics.confusion_matrix(y_true, y_pred, normalize='true'), annot=True, ax = ax, cmap = plt.cm.viridis);

ax.set_title('Confusion Matrix') 
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)

# Scenario 2
## Problem Statement
- **Theobjective  is  to  detect  Moving  Cars  inavideo  file using OpenCV using the HaarCascade_car.xml file and then, you will use OpenCV to detect the License plates of a Car using the HaarCascade_russian_plate_numberXMLfileTasks to be performed.**
- •Prepare a detailed python notebook using OpenCVto detect Moving Cars in a video
- •Import Required Libraries
- •Create a Classifier usingthe HaarCascade_car xml file
- •Load the Video 
- •Detect the Moving Cars using the Classifier
- •Display the Results


#### Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import cv2
from PIL import Image
np.random.seed(42)

#### Create a Classifier usingthe HaarCascade_car xml file

In [ ]:
# Load the cascade
car_cascade = cv2.CascadeClassifier(r"C:\Users\Shivani Dussa\Downloads\DL - MCT Data\HaarCascade_cars.xml")

#### Load the Video

In [ ]:
load_video = cv2.VideoCapture(r"C:\Users\Shivani Dussa\Downloads\Pexels Videos 2053100.mp4")

In [ ]:
def fixColor(image):
    return(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
from IPython.display import Video
Video(r"C:\Users\Shivani Dussa\Downloads\Pexels Videos 2053100.mp4",embed = True)

### Detect the Moving Cars using the Classifier

In [ ]:
while load_video.isOpened():
    
    time.sleep(.05)
    ret, frame = load_video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cars = car_cascade.detectMultiScale(gray, 1.4,2)
    
    #Display the results 
    for (x,y,w,h) in cars:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)
        cv2.imshow('Cars',frame)
    
    if cv2.waitKey(1) == 13:
        break
        
load_video.release()
cv2.destroyAllWindows()

In [ ]:
# Here I detected the cars in the vide to checks run this code u can the cars detection

### To detect the license plate of a car

### Import Required Libraries

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image

#### Load the Image using the imread function of OpenCV

In [ ]:
image = cv2.imread(r"C:\Users\Shivani Dussa\Downloads\cars.jpg")

In [ ]:
image2 = cv2.imread(r"C:\Users\Shivani Dussa\Downloads\car4.jpg")

#### Pre-process the Image

#### Create a function to convert the image using cv2.Color(img, cv2.COLOR_BGR2RGB) function

In [ ]:
converting_image = cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY)

In [ ]:
converting_image

#### Display the image

In [ ]:
imgplot = plt.imshow(image2)
plt.show()

In [ ]:
imgplot = plt.imshow(converting_image)
plt.show()

#### Create afunction for a classifier using the HaarCascade_russian_plate_number.xml file

In [ ]:
car_NumberPlate = cv2.CascadeClassifier(r"C:\Users\Shivani Dussa\Downloads\DL - MCT Data\haarcascade_russian_plate_number.xml")

#### Detect the Car License Plates using the Classifier

In [ ]:
faces = car_NumberPlate.detectMultiScale(converting_image,scaleFactor = 1.2, minNeighbors = 5,minSize = (25,25))
for (x,y,w,h) in faces:
    cv2.rectangle(converting_image,(x,y),(x+w,y+h),(255,0,0),2)
    plate = converting_image[y: y+h, x:x+w]
    plate = cv2.blur(plate,ksize=(20,20))
    
    # Display the results
    converting_image[y: y+h, x:x+w] = plate

cv2.imshow('plates',converting_image)
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()

## Thank You 